## Tarefa 1

### Modulo 24

Nome: Leandro Ferreira

______

#### 1 - Cite 5 diferenças entre o Random Forest e o AdaBoost

- O RF faz uso do bagging criando novas dataframes com linhas com reposição, enquanto o AdaBoost não.
- No RF cada árvore é independente e gera um resultado independente que não apresenta o peso da árvore. Já no AB, após a primeira stump (árvore de profundidade 1, 1 quebra e duas folhas), todas as demais serão dependentes da anterior numa espécie de efeito em cadeia.
- No AB todas as linhas sofrerão uma alteração do seu peso (w) seguido de uma normalização para então serem reutilizada na criação de uma nova dataframe com linhas amostrais que respeitam o peso de cada linha.
- No AB avalia-se a performance da árvore o que garante que cada árvore tenha um peso diferente. Já no RF todas as árvores têm o mesmo peso
- No RF é identificada a frequência na classificação das árvores e então aquela com maior frequência é o resultado final. No AB, o resultado final é obtido a partir do agrupamento binário entre as árvores que obtiveram um resultado de classificação semelhante. Então, é realizado um somatório das performance de cada grupo de classificação. Através do processo de votação ponderada, o somatório com maior resultado é tomado como o resultado final do processo.


#### 2 - Acesse o link Scikit-learn–adaboost, leia a explicação (traduza se for preciso) e crie um jupyter notebook contendo o exemplo do AdaBoost.

https://scikit-learn.org/stable/modules/ensemble.html#adaboost

In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

from scipy.stats import ks_2samp
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import patsy

In [3]:
X, y = load_iris(return_X_y=True)
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, X, y, cv=5)
scores.mean()

0.9466666666666665

#### 3 - Cite 5 Hyperparametros importantes no AdaBoost

- 1. ```n_estimators```: int, default=50 (1 - inf)- controla a quantidade de weak learners.
-  2. ```learning_rate```: float, default=1.0 (0.0 - inf) - o peso aplicado a cada classificador em cada iteração do boosting. Uma alta taxa de aprendizagem
    aumenta a contribuição de cada classificador. Existe
    uma correlação entre os parâmetros ```learning_rate``` e ```n_estimators```.
- 3. ```algorithm```: {'SAMME', 'SAMME.R'}, default='SAMME.R' - definição de qual algorítmo boosting pretende usar.
- 4. ```random_state```: Controla a aleatoriedade das amostras.
- 5. ```estimator```: object, default=None - O estimador de base a partir do qual o conjunto otimizado é construído. ```base_estimator``` não é mais utilizado 

#### 4 - (Opcional) Utilize o GridSearch para encontrar os melhores hyperparametros para o conjunto de dados do exemplo (load_iris)

In [30]:
df = load_iris(as_frame=True)
df = pd.DataFrame(df.data, columns=df.feature_names)
df['target'] = y
df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [45]:
df.columns 

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [78]:
df.target.value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

In [58]:
# por alguma razão que não sei explicar, o patsy não conseguia usar o método dmatrices 
# pq esbarrava um erro de syntax que acusava a variável petal length (cm) de não estar escrita na forma correta ???
# daí alterei o nome das colunas para algo genérico pra ver se dava certo e deu
df1 = df.copy()
df1.columns = ['A','B','C','D','target']
df1.head()

,A,B,C,D,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [69]:
df_train, df_test = train_test_split(df1, test_size=0.3, random_state=12) #usei a df1, a cópia da df originalb

In [70]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [71]:
# construindo a equação
equacao = 'target ~ A + B + C + D'
           
y_train, X_train = patsy.dmatrices(equacao, data=df_train)
y_test, X_test = patsy.dmatrices(equacao, data=df_test)

Se bem entendi os parâmetros do ```AdaBoostClassifier```, aqueles que de fato podem fazer alguma diferença para melhorar o modelo de classificação seriam o ```learning_rate``` e ```n_estimators```. Portanto, como tentativa de utiliza-los num gridsearch, criarei uma lista somente para alterá-los

In [72]:
grid_lr = [1,50,100, 500, 1000, 5000] # grid para o learning rate
grid_ne = [50, 100, 500, 1000, 5000] #grid para o n estimator

In [81]:
# Calcular o Gini
def calcula_gini(RESP, PD):
    #AUC
    auc = roc_auc_score(RESP, PD,multi_class = 'ovo')
    
    #Gini
    gini = 2*auc -1
    return gini


def print_metricas(dados, 
                  PD = 'PD', 
                  CLASSE_PRED = 'classe_predita', 
                  RESP = 'mau'):
    
    #Acuracia
    acc = accuracy_score(dados[RESP], dados[CLASSE_PRED])

    #AUC
    auc = roc_auc_score(dados[RESP], dados[PD])
    
    #Gini
    gini = 2*auc-1
    
    #KS
    ks = ks_2samp(dados.loc[dados[RESP] == 1, PD], 
                  dados.loc[dados[RESP] != 1, PD]).statistic

    print('KS:       {0:.2f}%'.format(ks*100))
    print('AUC:      {0:.2f}%'.format(auc*100))
    print('GINI:     {0:.2f}%'.format(gini*100))
    print('Acurácia: {0:.2f}%\n'.format(acc*100))
    
    return None

In [74]:
lista_gini_test=[]
lista_gini_train=[]

In [82]:
%%time

for rate in grid_lr:
    clf = AdaBoostClassifier(learning_rate=rate)
    
    clf.fit(X_train,y_train.ravel())
    
    gini_test  = calcula_gini(y_test, clf.predict_proba(X_test)[:,1])
    gini_train = calcula_gini(y_train, clf.predict_proba(X_train)[:,1])
    
    lista_gini_test.append(gini_test)
    lista_gini_train.append(gini_train)

AxisError: axis 1 is out of bounds for array of dimension 1

> Por alguma razão, que tentei entender e resolver que tentei resolver, mas não consegui, fiquei travado nesta linha.